<a href="https://colab.research.google.com/github/GalBuzi/colab/blob/main/Ass2Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
%matplotlib inline

In [ ]:
foler_data_path = r'drive/My Drive/DeepLearnCourse/ass2/Protocol/'

subjectID = [101,102,103,104,105,106,107,108,109]

activityIDdict = {0: 'transient',
              1: 'lying',
              2: 'sitting',
              3: 'standing',
              4: 'walking',
              5: 'running',
              6: 'cycling',
              7: 'Nordic_walking',
              9: 'watching_TV',
              10: 'computer_work',
              11: 'car driving',
              12: 'ascending_stairs',
              13: 'descending_stairs',
              16: 'vacuum_cleaning',
              17: 'ironing',
              18: 'folding_laundry',
              19: 'house_cleaning',
              20: 'playing_soccer',
              24: 'rope_jumping' }

cols = ["timestamp", "activityID","heartrate"]

IMUhand = ['handTemperature', 
           'handAcc16_x', 'handAcc16_y', 'handAcc16_z', 
           'handAcc6_x', 'handAcc6_y', 'handAcc6_z', 
           'handGyro_x', 'handGyro_y', 'handGyro_z', 
           'handMagne_x', 'handMagne_y', 'handMagne_z',
           'handOrientation1', 'handOrientation2', 'handOrientation3', 'handOrientation4']


IMUchest = ['chestTemperature', 
           'chestAcc16_x', 'chestAcc16_y', 'chestAcc16_z', 
           'chestAcc6_x', 'chestAcc6_y', 'chestAcc6_z', 
           'chestGyro_x', 'chestGyro_y', 'chestGyro_z', 
           'chestMagne_x', 'chestMagne_y', 'chestMagne_z',
           'chestOrientation1', 'chestOrientation2', 'chestOrientation3', 'chestOrientation4']


IMUankle = ['ankleTemperature', 
           'ankleAcc16_x', 'ankleAcc16_y', 'ankleAcc16_z', 
           'ankleAcc6_x', 'ankleAcc6_y', 'ankleAcc6_z', 
           'ankleGyro_x', 'ankleGyro_y', 'ankleGyro_z', 
           'ankleMagne_x', 'ankleMagne_y', 'ankleMagne_z',
           'ankleOrientation1', 'ankleOrientation2', 'ankleOrientation3', 'ankleOrientation4']

columns = cols + IMUhand + IMUchest + IMUankle 
columns_with_subject = cols + IMUhand + IMUchest + IMUankle + ['subject_num']

len(columns) # need to be 54

we need to read all train data into one dataframe to work with
we will read each subject table and concatenate them to eachother

# Q1 - Data Exploration:

In [ ]:
all_data = pd.DataFrame()
for i in range (1, 10):
    procData = pd.read_table(foler_data_path + 'subject10{}.dat'.format(i), header=None, sep='\s+')
    procData.columns = columns
    procData['subject_id'] = 100 + i
    all_data = all_data.append(procData, ignore_index=True)

all_data.reset_index(drop=True, inplace=True)
all_data.head()


In [ ]:
print('Size of the data: ', all_data.size)
print('Shape of the data: ', all_data.shape)
print('Number of columns in the data: ', len(all_data.columns))
result_id = all_data.groupby(['subject_id']).mean().reset_index()
print('Number of uniqe ids in the data: ', len(result_id))
result_act = all_data.groupby(['activityID']).mean().reset_index()
print('Numbe of uniqe activitys in the data: ',len(result_act))

as we can learn from the data above, the data must go through some cleaning, for example:

removing any activity with '0' because this activity means the subject isn't doing any from the activities that were mentioned in the README file.

it has very noisy data and that affects the output because the heartrate values will not be extracted from the data point set in the actual important activities.


In [ ]:
#clean data
all_data = all_data.drop(all_data[all_data['activityID']==0].index)
all_data = all_data.interpolate()
# fill all the NaN values in a coulmn with the mean values of the column
for colName in all_data.columns:
    all_data[colName] = all_data[colName].fillna(all_data[colName].mean())
activity_mean = all_data.groupby(['activityID']).mean().reset_index()

all_data.shape

In [ ]:
def plot_samples(df, column_a, column_b, title, figsize=(10,6)):
    plt.rcParams.update({'font.size': 16})
    size = range(len(df))
    f, ax = plt.subplots(figsize=figsize) 
    plt.bar(size, df[column_a])
    a = ax.set_xticklabels(df[column_b])
    b = ax.legend(fontsize = 20)
    c = ax.set_xticks(np.arange(len(df)))
    d = ax.set_title(title)
    plt.show()

In [ ]:
#present num of samples per subject
sampels = all_data.groupby(['subject_id']).count().reset_index()
sampels_to_subject = pd.DataFrame()
sampels_to_subject['id'] = sampels['subject_id']
sampels_to_subject['sampels'] = sampels['timestamp']
sampels_to_subject = sampels_to_subject.sort_values(by=['sampels'])
plot_samples(sampels_to_subject,'sampels','id','Number Of Samepls Per Subject')

data isn't balanced for samples per subject, subject 109 has very little samples, we will probably use him as validation to check our model

In [ ]:
sampels = all_data.groupby(['activityID']).count().reset_index()
sampels_to_subject = pd.DataFrame()
sampels_to_subject['activity'] = [activityIDdict[x] for x in sampels['activityID']]
sampels_to_subject['sampels'] = sampels['timestamp']
sampels_to_subject = sampels_to_subject.sort_values(by=['sampels'])
plot_samples(sampels_to_subject,'sampels','activity','Number Of Samepls Per Activity',figsize=(50,10))
sampels_to_subject = None

rope jumping has significally less samples - the data isn't balanced again

In [ ]:
sampels_heart_rate = pd.DataFrame()
sampels_heart_rate['id'] = result_id['subject_id']
sampels_heart_rate['heartrate'] = result_id['heartrate']
sampels_heart_rate = sampels_heart_rate.sort_values(by=['heartrate'])
plot_samples(sampels_heart_rate,'heartrate','id','Avg heart Rate Per Subject')

In [ ]:
sampels_heart_rate = pd.DataFrame()
sampels_heart_rate['activity'] = [activityIDdict[x] for x in result_act['activityID']]
sampels_heart_rate['heartrate'] = result_act['heartrate']
sampels_heart_rate = sampels_heart_rate.sort_values(by=['heartrate'])
plot_samples(sampels_heart_rate,'heartrate','activity','Avg heart Rate Per Activity',figsize=(50,10))
sampels_heart_rate = None

In [ ]:
samepls_tempreture = pd.DataFrame()
samepls_tempreture['hand'] = result_id['handTemperature']
samepls_tempreture['chest'] = result_id['chestTemperature']
samepls_tempreture['ankle'] = result_id['ankleTemperature']

ax = samepls_tempreture.plot(kind='line', figsize=(20,6), title='Avg Tempatures by Subjects')
a = ax.set_xticklabels(result_id['subject_id'])
b = ax.legend(fontsize = 20)
c = ax.set_xticks(np.arange(len(samepls_tempreture)))
samepls_tempreture=None

In [ ]:
samepls_tempreture = pd.DataFrame()
samepls_tempreture['activity'] = [activityIDdict[x] for x in result_act['activityID']]
samepls_tempreture['hand'] = result_act['handTemperature']
samepls_tempreture['chest'] = result_act['chestTemperature']
samepls_tempreture['ankle'] = result_act['ankleTemperature']

ax = samepls_tempreture.plot(kind='line', figsize=(50,6), title='Avg Tempatures by Activity')
a = ax.set_xticklabels(samepls_tempreture['activity'])
b = ax.legend(fontsize = 20)
c = ax.set_xticks(np.arange(len(samepls_tempreture)))

In [ ]:
samepls = pd.DataFrame()
samepls['hand_x'] = result_id['handAcc16_x']
samepls['hand_y'] = result_id['handAcc16_y']
samepls['hand_z'] = result_id['handAcc16_z']
ax = samepls.plot(kind='line', figsize=(20,6), title='Avg Hand Acceleration Value by Subjects')
a = ax.set_xticklabels(result_id['subject_id'])
b = ax.legend(fontsize = 20)
c = ax.set_xticks(np.arange(len(samepls)))

In [ ]:
samepls = pd.DataFrame()
samepls['chest_x'] = result_id['chestAcc16_x']
samepls['chest_y'] = result_id['chestAcc16_y']
samepls['chest_z'] = result_id['chestAcc16_z']
ax = samepls.plot(kind='line', figsize=(20,6), title='Avg Chest Acceleration Value by Subjects')
a = ax.set_xticklabels(result_id['subject_id'])
b = ax.legend(fontsize = 20)
c = ax.set_xticks(np.arange(len(samepls)))

In [ ]:
samepls = pd.DataFrame()
samepls['ankle_x'] = result_id['ankleAcc16_x']
samepls['ankle_y'] = result_id['ankleAcc16_y']
samepls['ankle_z'] = result_id['ankleAcc16_z']
ax = samepls.plot(kind='line', figsize=(20,6), title='Avg Ankle Acceleration Value by Subjects')
a = ax.set_xticklabels(result_id['subject_id'])
b = ax.legend(fontsize = 20)
c = ax.set_xticks(np.arange(len(samepls)))
samepls = None

In [ ]:
!pip install heatmapz

In [ ]:
from heatmap import heatmap, corrplot
dropped_activity = all_data.drop(['activityID'], axis = 1)

plt.figure(figsize=(20, 20))
corrplot(dropped_activity.corr(), size_scale=300);
dropped_activity = None

# Q2 :

we will use for train set subjects 101-106 and 109 .

subjects 107-108 will be our test set.

In [ ]:
# create the test data
subject107 = all_data[all_data['subject_id'] == 107]
subject108 = all_data[all_data['subject_id'] == 108]
test = subject107.append(subject108)

# create the train data
train = all_data[all_data['subject_id'] != 107]
train = all_data[all_data['subject_id'] != 108]
all_data = None

# drop the columns id
test = test.drop(["subject_id"], axis=1) # 107-108
train = train.drop(["subject_id"], axis=1) # all the rest

# split train and test to X and y
X_train = train.drop(['activity_id'], axis=1).values
X_test = test.drop(['activity_id'], axis=1).values
y_train = train['activity_id'].values
y_test = test['activity_id'].values

# # make data scale to min max beetwin 0 to 1
# min_max_scaler = MinMaxScaler()
# min_max_scaler.fit(X_train)
# min_max_scaler.fit(X_test)
# X_train = min_max_scaler.transform(X_train)
# X_test = min_max_scaler.transform(X_test)

